In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_69940/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 13:42:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/21 13:42:50 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Sessions table
sessions_data = {
    'user_id': [101, 101, 102, 102, 101, 102, 101, 102, 103, 103, 103],
    'session_start': ['2023-11-01 08:00:00', '2023-11-01 10:00:00', '2023-11-01 13:00:00', '2023-11-01 15:00:00',
                      '2023-11-02 09:00:00', '2023-11-02 12:00:00', '2023-11-02 13:00:00', '2023-11-02 16:00:00',
                      '2023-11-01 08:00:00', '2023-11-02 20:00:00', '2023-11-03 09:00:00'],
    'session_end': ['2023-11-01 09:00:00', '2023-11-01 11:00:00', '2023-11-01 14:00:00', '2023-11-01 16:00:00',
                    '2023-11-02 10:00:00', '2023-11-02 13:00:00', '2023-11-02 14:00:00', '2023-11-02 17:00:00',
                    '2023-11-01 09:00:00', '2023-11-02 23:00:00', '2023-11-03 10:00:00'],
    'session_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'session_type': ['Viewer', 'Streamer', 'Viewer', 'Viewer', 'Viewer', 'Streamer', 'Streamer', 'Viewer', 'Viewer', 'Viewer', 'Viewer']
}

# Create DataFrame for Sessions
sessions_df = pd.DataFrame(sessions_data)

# Display the DataFrame
print("Sessions table:")
print(sessions_df)



df_person = spark.createDataFrame(sessions_df)
df_person.createOrReplaceTempView("Sessions")



Sessions table:
    user_id        session_start          session_end  session_id session_type
0       101  2023-11-01 08:00:00  2023-11-01 09:00:00           1       Viewer
1       101  2023-11-01 10:00:00  2023-11-01 11:00:00           2     Streamer
2       102  2023-11-01 13:00:00  2023-11-01 14:00:00           3       Viewer
3       102  2023-11-01 15:00:00  2023-11-01 16:00:00           4       Viewer
4       101  2023-11-02 09:00:00  2023-11-02 10:00:00           5       Viewer
5       102  2023-11-02 12:00:00  2023-11-02 13:00:00           6     Streamer
6       101  2023-11-02 13:00:00  2023-11-02 14:00:00           7     Streamer
7       102  2023-11-02 16:00:00  2023-11-02 17:00:00           8       Viewer
8       103  2023-11-01 08:00:00  2023-11-01 09:00:00           9       Viewer
9       103  2023-11-02 20:00:00  2023-11-02 23:00:00          10       Viewer
10      103  2023-11-03 09:00:00  2023-11-03 10:00:00          11       Viewer


In [6]:
query = """
    select 
        *
    from Sessions s where user_id = 101
    order by session_start
"""

In [7]:
spark.sql(query).show()

+-------+-------------------+-------------------+----------+------------+
|user_id|      session_start|        session_end|session_id|session_type|
+-------+-------------------+-------------------+----------+------------+
|    101|2023-11-01 08:00:00|2023-11-01 09:00:00|         1|      Viewer|
|    101|2023-11-01 10:00:00|2023-11-01 11:00:00|         2|    Streamer|
|    101|2023-11-02 09:00:00|2023-11-02 10:00:00|         5|      Viewer|
|    101|2023-11-02 13:00:00|2023-11-02 14:00:00|         7|    Streamer|
+-------+-------------------+-------------------+----------+------------+



In [10]:
spark.sql("""
    select 
        *
    from Sessions s where user_id = 102
    order by session_start
""").show()

spark.sql("""
    select 
        *
    from Sessions s where user_id = 103
    order by session_start
""").show()

+-------+-------------------+-------------------+----------+------------+
|user_id|      session_start|        session_end|session_id|session_type|
+-------+-------------------+-------------------+----------+------------+
|    102|2023-11-01 13:00:00|2023-11-01 14:00:00|         3|      Viewer|
|    102|2023-11-01 15:00:00|2023-11-01 16:00:00|         4|      Viewer|
|    102|2023-11-02 12:00:00|2023-11-02 13:00:00|         6|    Streamer|
|    102|2023-11-02 16:00:00|2023-11-02 17:00:00|         8|      Viewer|
+-------+-------------------+-------------------+----------+------------+

+-------+-------------------+-------------------+----------+------------+
|user_id|      session_start|        session_end|session_id|session_type|
+-------+-------------------+-------------------+----------+------------+
|    103|2023-11-01 08:00:00|2023-11-01 09:00:00|         9|      Viewer|
|    103|2023-11-02 20:00:00|2023-11-02 23:00:00|        10|      Viewer|
|    103|2023-11-03 09:00:00|2023-11-

In [22]:
spark.sql("""
    select
        distinct user_id
    from (
        select
            *,
            sum(
                case
                    when is_conse then 0
                    else 1
                end
            ) over(partition by user_id order by session_start asc) as group_id
        from (
            select
                *,
                (is_same_session_type and is_less_than_12) as is_conse
            from (
                select 
                    *,
                    ifnull(
                        session_type = lag(session_type) over(partition by user_id order by session_start asc), 
                        False
                    ) as is_same_session_type,
                    ifnull(
                        cast(to_timestamp(session_start) as long) 
                        -
                        cast(
                            to_timestamp(
                                lag(session_end) over(partition by user_id order by session_start asc)
                            ) as long
                        ), 
                        0
                    ) <= 12 * 60 * 60 as is_less_than_12
                from Sessions s
            )
        )
    ) group by user_id, group_id having count(*) >= 2
    


""").show()

+-------+
|user_id|
+-------+
|    102|
|    103|
+-------+

